### Configure Model

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', input_shape=(256,256,3), activation="relu"))
model.add(Conv2D(64, (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(128, (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(4, 4)))         #(2,2) ?
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dense(16,activation="relu"))
model.add(Dense(1, activation='relu'))
model.compile(optimizer="adam",loss="mean_squared_error",metrics=["accuracy"])
print('Model compiled successfully!')
model.save(filepath="/storage/Manual Model.h5",
               overwrite=True,include_optimizer=True,save_format='h5')
print("Model saved successfully!")



model.compile(loss = 'mean_squared_error', optimizer='adam')

Model compiled successfully!
Model saved successfully!


### Configure Data Generator

In [5]:
import pandas as pd
df = pd.read_csv("/storage/data.csv")
df['filename']="clips-"+df["id"].astype(str)+".png"

df_validate = pd.read_csv("/storage/validate.csv", na_values=['NA', '?'])
df_validate['filename']="clips-"+df_validate["id"].astype(str)+".png"


df_train = df[:40000] #45000 images in total
df_test = df[40000:45000]

import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

IMAGES_DIR = "/storage/clips"

training_datagen = ImageDataGenerator(rescale = 1./255., horizontal_flip=True, 
                                      vertical_flip=True, fill_mode='nearest')
train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory=IMAGES_DIR,
        x_col="filename",
        y_col="clip_count",
        target_size=(256, 256),
        batch_size=8,
        class_mode='other')

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=IMAGES_DIR,
        x_col="filename",
        y_col="clip_count",
        target_size=(256, 256),
        class_mode='other')

Found 40000 validated image filenames.
Found 5000 validated image filenames.


In [6]:
import pandas as pd
import numpy as np
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
model=load_model("/storage/Manual Model.h5",compile=False)
model.compile(optimizer="adam",loss="mean_squared_error")
print("Successfully loaded h5 model!")

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
print("Successfully created callback: Monitor")
checkpoint = ModelCheckpoint("/storage/Manual Model.h5", 
                             monitor='val_loss', verbose=1)
print("Successfully created callback: Checkpoint")
history = model.fit_generator(train_generator, validation_data=test_generator, steps_per_epoch=5000,
                    callbacks=[checkpoint,monitor], epochs=50, verbose = 1,
                    initial_epoch=0) 


Successfully loaded h5 model!
Successfully created callback: Monitor
Successfully created callback: Checkpoint
Epoch 1/50
4999/5000 [============================>.] - ETA: 0s - loss: 9.8017
Epoch 00001: saving model to /storage/Manual Model.h5
5000/5000 [==============================] - 742s 148ms/step - loss: 9.8022 - val_loss: 3.0160
Epoch 2/50
4999/5000 [============================>.] - ETA: 0s - loss: 3.5799
Epoch 00002: saving model to /storage/Manual Model.h5
5000/5000 [==============================] - 720s 144ms/step - loss: 3.5797 - val_loss: 2.7345
Epoch 3/50
4999/5000 [============================>.] - ETA: 0s - loss: 2.7049
Epoch 00003: saving model to /storage/Manual Model.h5
5000/5000 [==============================] - 715s 143ms/step - loss: 2.7046 - val_loss: 2.1451
Epoch 4/50
4999/5000 [============================>.] - ETA: 0s - loss: 2.2991
Epoch 00004: saving model to /storage/Manual Model.h5
5000/5000 [==============================] - 769s 154ms/step - loss: 2.2

In [7]:
from tensorflow.keras.models import load_model
model=load_model("/storage/2020-04-06(0.61).h5",compile=False)
model.compile(optimizer="adam",loss="mean_squared_error")
print("Successfully loaded h5 model!")

Successfully loaded h5 model!


In [8]:
import pandas as pd

df_validate = pd.read_csv("/storage/validate.csv", na_values=['NA', '?'])
df_validate['filename']="clips-"+df_validate["id"].astype(str)+".png"

import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

IMAGES_DIR = "/storage/clips"

val_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = val_datagen.flow_from_dataframe(
        dataframe=df_validate,
        directory=IMAGES_DIR,
        x_col="filename",
        y_col="id",
        target_size=(256, 256),
        class_mode='other')

Found 5000 validated image filenames.


In [12]:
from PIL import Image
import numpy as np
from tqdm import tqdm

df = pd.DataFrame()

for i in tqdm(range(5000)):
    arr=np.expand_dims(np.array(Image.open(f"/storage/clips/clips-{25001+i}.png"))/255, 0)
    df.loc[i+25001,"clips"]=model.predict(arr)
df

100%|██████████| 5000/5000 [03:48<00:00, 21.86it/s]


,clips
25001,2.008205
25002,47.523621
25003,64.273804
25004,3.162733
25005,45.325871
...,...
29996,32.216011
29997,53.702774
29998,61.293934
29999,4.634710


In [13]:
df.to_csv("/notebooks/submit.csv")

In [10]:
! pip install tqdm

     |████████████████████████████████| 61kB 7.5MB/s eta 0:00:011
You should consider upgrading via the 'pip install --upgrade pip' command.
